In [1]:
import re
from typing import Dict, List, Optional, Text, Tuple
import matplotlib.pyplot as plt
from matplotlib import colors

import tensorflow as tf

2025-03-30 16:21:08.812013: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Download the dataset from kaggle and add it to a folder called "data" in the directory. Make sure not to commit this to the repository (it's included in the .gitignore already)

In [12]:
data_pattern_train = 'data/next_day_wildfire_spread_train*'
data_pattern_eval = 'data/next_day_wildfire_spread_eval*'
data_pattern_test = 'data/next_day_wildfire_spread_test*'

In [ ]:
"""Constants for the data reader."""

INPUT_FEATURES = ['elevation', 'th', 'vs',  'tmmn', 'tmmx', 'sph', 
                  'pr', 'pdsi', 'NDVI', 'population', 'erc','temperature_2m_above_ground','u_component_of_wind_10m_above_ground','v_component_of_wind_10m_above_ground','precipitable_water_entire_atmosphere', 'PrevFireMask']

OUTPUT_FEATURES = ['FireMask', ]

# Data statistics 
# For each variable, the statistics are ordered in the form:
# (min_clip, max_clip, mean, standard deviation)
DATA_STATS = {
    # Elevation in m.
    # 0.1 percentile, 99.9 percentile
    'elevation': (0.0, 3141.0, 657.3003, 649.0147),
    
    # Drought Index (Palmer Drought Severity Index)
    # 0.1 percentile, 99.9 percentile
    'pdsi': (-6.12974870967865, 7.876040384292651, -0.0052714925, 2.6823447),
    
    #Vegetation index (times 10,000 maybe, since it's supposed to be b/w -1 and 1?)
    'NDVI': (-9821.0, 9996.0, 5157.625, 2466.6677),  # min, max
   
    # Precipitation in mm.
    # Negative values do not make sense, so min is set to 0.
    # 0., 99.9 percentile
    'pr': (0.0, 44.53038024902344, 1.7398051, 4.482833),
   
    # Specific humidity.
    # Negative values do not make sense, so min is set to 0.
    # The range of specific humidity is up to 100% so max is 1.
    'sph': (0., 1., 0.0071658953, 0.0042835088),
    
    # Wind direction in degrees clockwise from north.
    # Thus min set to 0 and max set to 360.
    'th': (0., 360.0, 190.32976, 72.59854),
    
    # Min/max temperature in Kelvin.
    
    #Min temp
    # -20 degree C, 99.9 percentile
    'tmmn': (253.15, 298.94891357421875, 281.08768, 8.982386),
    
    #Max temp
    # -20 degree C, 99.9 percentile
    'tmmx': (253.15, 315.09228515625, 295.17383, 9.815496),
    
    # Wind speed in m/s.
    # Negative values do not make sense, given there is a wind direction.
    # 0., 99.9 percentile
    'vs': (0.0, 10.024310074806237, 3.8500874, 1.4109988),
    
    # NFDRS fire danger index energy release component expressed in BTU's per
    # square foot.
    # Negative values do not make sense. Thus min set to zero.
    # 0., 99.9 percentile
    'erc': (0.0, 106.24891662597656, 37.326267, 20.846027),
    
    # Population density
    # min, 99.9 percentile
    'population': (0., 2534.06298828125, 25.531384, 154.72331),

    #FORECAST DATA - ADDED
    #TO DO: Calculate min, max, mean, standard deviation for forecast variables
    'temperature_2m_above_ground': (),

    'u_component_of_wind_10m_above_ground': (),

    'v_component_of_wind_10m_above_ground': (),
        
    'precipitable_water_entire_atmosphere': (),
    
    # We don't want to normalize the FireMasks.
    # 1 indicates fire, 0 no fire, -1 unlabeled data
    'PrevFireMask': (-1., 1., 0., 1.),
    'FireMask': (-1., 1., 0., 1.)
}

In [4]:
"""Library of common functions used in deep learning neural networks.
"""
#YOU PROBABLY WILL NOT USE THESE.

def random_crop_input_and_output_images(
    input_img: tf.Tensor,
    output_img: tf.Tensor,
    sample_size: int,
    num_in_channels: int,
    num_out_channels: int,
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Randomly axis-align crop input and output image tensors.

  Args:
    input_img: tensor with dimensions HWC.
    output_img: tensor with dimensions HWC.
    sample_size: side length (square) to crop to.
    num_in_channels: number of channels in input_img.
    num_out_channels: number of channels in output_img.
  Returns:
    input_img: tensor with dimensions HWC.
    output_img: tensor with dimensions HWC.
  """
  combined = tf.concat([input_img, output_img], axis=2)
  combined = tf.image.random_crop(
      combined,
      [sample_size, sample_size, num_in_channels + num_out_channels])
  input_img = combined[:, :, 0:num_in_channels]
  output_img = combined[:, :, -num_out_channels:]
  return input_img, output_img


def center_crop_input_and_output_images(
    input_img: tf.Tensor,
    output_img: tf.Tensor,
    sample_size: int,
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Center crops input and output image tensors.

  Args:
    input_img: tensor with dimensions HWC.
    output_img: tensor with dimensions HWC.
    sample_size: side length (square) to crop to.
  Returns:
    input_img: tensor with dimensions HWC.
    output_img: tensor with dimensions HWC.
  """
  central_fraction = sample_size / input_img.shape[0]
  input_img = tf.image.central_crop(input_img, central_fraction)
  output_img = tf.image.central_crop(output_img, central_fraction)
  return input_img, output_img

In [5]:
"""Dataset reader for Earth Engine data."""

def _get_base_key(key: Text) -> Text:
  """Extracts the base key from the provided key.

  Earth Engine exports TFRecords containing each data variable with its
  corresponding variable name. In the case of time sequences, the name of the
  data variable is of the form 'variable_1', 'variable_2', ..., 'variable_n',
  where 'variable' is the name of the variable, and n the number of elements
  in the time sequence. Extracting the base key ensures that each step of the
  time sequence goes through the same normalization steps.
  The base key obeys the following naming pattern: '([a-zA-Z]+)'
  For instance, for an input key 'variable_1', this function returns 'variable'.
  For an input key 'variable', this function simply returns 'variable'.

  Args:
    key: Input key.

  Returns:
    The corresponding base key.

  Raises:
    ValueError when `key` does not match the expected pattern.
  """
  match = re.match(r'([a-zA-Z]+)', key)
  if match:
    return match.group(1)
  raise ValueError(
      'The provided key does not match the expected pattern: {}'.format(key))


def _clip_and_rescale(inputs: tf.Tensor, key: Text) -> tf.Tensor:
  """Clips and rescales inputs with the stats corresponding to `key`.

  Args:
    inputs: Inputs to clip and rescale.
    key: Key describing the inputs.

  Returns:
    Clipped and rescaled input.

  Raises:
    ValueError if there are no data statistics available for `key`.
  """
  base_key = _get_base_key(key)
  if base_key not in DATA_STATS:
    raise ValueError(
        'No data statistics available for the requested key: {}.'.format(key))
  min_val, max_val, _, _ = DATA_STATS[base_key]
  inputs = tf.clip_by_value(inputs, min_val, max_val)
  return tf.math.divide_no_nan((inputs - min_val), (max_val - min_val))


def _clip_and_normalize(inputs: tf.Tensor, key: Text) -> tf.Tensor:
  """Clips and normalizes inputs with the stats corresponding to `key`.

  Args:
    inputs: Inputs to clip and normalize.
    key: Key describing the inputs.

  Returns:
    Clipped and normalized input.

  Raises:
    ValueError if there are no data statistics available for `key`.
  """
  base_key = _get_base_key(key)
  if base_key not in DATA_STATS:
    raise ValueError(
        'No data statistics available for the requested key: {}.'.format(key))
  min_val, max_val, mean, std = DATA_STATS[base_key]
  inputs = tf.clip_by_value(inputs, min_val, max_val)
  inputs = inputs - mean
  return tf.math.divide_no_nan(inputs, std)

def _get_features_dict(
    sample_size: int,
    features: List[Text],
) -> Dict[Text, tf.io.FixedLenFeature]:
  """Creates a features dictionary for TensorFlow IO.

  Args:
    sample_size: Size of the input tiles (square).
    features: List of feature names.

  Returns:
    A features dictionary for TensorFlow IO.
  """
  sample_shape = [sample_size, sample_size]
  features = set(features)
  columns = [
      tf.io.FixedLenFeature(shape=sample_shape, dtype=tf.float32)
      for _ in features
  ]
  return dict(zip(features, columns))


def _parse_fn(
    example_proto: tf.train.Example, data_size: int, sample_size: int,
    num_in_channels: int, clip_and_normalize: bool,
    clip_and_rescale: bool, random_crop: bool, center_crop: bool,
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Reads a serialized example.

  Args:
    example_proto: A TensorFlow example protobuf.
    data_size: Size of tiles (square) as read from input files.
    sample_size: Size the tiles (square) when input into the model.
    num_in_channels: Number of input channels.
    clip_and_normalize: True if the data should be clipped and normalized.
    clip_and_rescale: True if the data should be clipped and rescaled.
    random_crop: True if the data should be randomly cropped.
    center_crop: True if the data should be cropped in the center.

  Returns:
    (input_img, output_img) tuple of inputs and outputs to the ML model.
  """
  if (random_crop and center_crop):
    raise ValueError('Cannot have both random_crop and center_crop be True')
  input_features, output_features = INPUT_FEATURES, OUTPUT_FEATURES
  feature_names = input_features + output_features
  features_dict = _get_features_dict(data_size, feature_names)
  features = tf.io.parse_single_example(example_proto, features_dict)

  if clip_and_normalize:
    inputs_list = [
        _clip_and_normalize(features.get(key), key) for key in input_features
    ]
  elif clip_and_rescale:
    inputs_list = [
        _clip_and_rescale(features.get(key), key) for key in input_features
    ]
  else:
    inputs_list = [features.get(key) for key in input_features]
  
  inputs_stacked = tf.stack(inputs_list, axis=0)
  input_img = tf.transpose(inputs_stacked, [1, 2, 0])

  outputs_list = [features.get(key) for key in output_features]
  assert outputs_list, 'outputs_list should not be empty'
  outputs_stacked = tf.stack(outputs_list, axis=0)

  outputs_stacked_shape = outputs_stacked.get_shape().as_list()
  assert len(outputs_stacked.shape) == 3, ('outputs_stacked should be rank 3'
                                            'but dimensions of outputs_stacked'
                                            f' are {outputs_stacked_shape}')
  output_img = tf.transpose(outputs_stacked, [1, 2, 0])

  if random_crop:
    input_img, output_img = random_crop_input_and_output_images(
        input_img, output_img, sample_size, num_in_channels, 1)
  if center_crop:
    input_img, output_img = center_crop_input_and_output_images(
        input_img, output_img, sample_size)
  return input_img, output_img


def get_dataset(file_pattern: Text, data_size: int, sample_size: int,
                batch_size: int, num_in_channels: int, compression_type: Text,
                clip_and_normalize: bool, clip_and_rescale: bool,
                random_crop: bool, center_crop: bool) -> tf.data.Dataset:
  """Gets the dataset from the file pattern.

  Args:
    file_pattern: Input file pattern.
    data_size: Size of tiles (square) as read from input files.
    sample_size: Size the tiles (square) when input into the model.
    batch_size: Batch size.
    num_in_channels: Number of input channels.
    compression_type: Type of compression used for the input files.
    clip_and_normalize: True if the data should be clipped and normalized, False
      otherwise.
    clip_and_rescale: True if the data should be clipped and rescaled, False
      otherwise.
    random_crop: True if the data should be randomly cropped.
    center_crop: True if the data shoulde be cropped in the center.

  Returns:
    A TensorFlow dataset loaded from the input file pattern, with features
    described in the constants, and with the shapes determined from the input
    parameters to this function.
  """
  if (clip_and_normalize and clip_and_rescale):
    raise ValueError('Cannot have both normalize and rescale.')
  dataset = tf.data.Dataset.list_files(file_pattern)
  dataset = dataset.interleave(
      lambda x: tf.data.TFRecordDataset(x, compression_type=compression_type),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  dataset = dataset.map(
      lambda x: _parse_fn(  # pylint: disable=g-long-lambda
          x, data_size, sample_size, num_in_channels, clip_and_normalize,
          clip_and_rescale, random_crop, center_crop),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return dataset

In [14]:
side_length = 32 #length of the side of the square you select (so, e.g. pick 64 if you don't want any random cropping)
num_obs = 100 #batch size

train_dataset = get_dataset(
      data_pattern_train,
      data_size=64,
      sample_size=side_length,
      batch_size=num_obs,
      num_in_channels=12,
      compression_type=None,
      clip_and_normalize=False,
      clip_and_rescale=False,
      random_crop=True,
      center_crop=False)

In [16]:
inputs, labels = next(iter(train_dataset)) 
#Are there two assignments happening on every iteration because dataset stores inputs with labels?
#print(inputs.shape) #(100, 32, 32, 12)
#print(labels.shape) #(100, 32, 32, 1)
#print(inputs[0, :, :, 11]) #Trying to grab the previous fire mask. (Apparent) success!
#print(labels[0,:, :, 0]) #Ok, I think the labels are the fire mask. (That also accords with standard usage of the term.)

In [17]:
inputs

<tf.Tensor: shape=(100, 32, 32, 12), dtype=float32, numpy=
array([[[[ 1.66900000e+03,  2.26539780e+02,  3.41282010e+00, ...,
           1.51426941e-01,  6.21819992e+01,  0.00000000e+00],
         [ 1.77500000e+03,  2.26431961e+02,  3.41747022e+00, ...,
           1.05957359e-01,  6.20866051e+01,  0.00000000e+00],
         [ 1.89300000e+03,  2.26339478e+02,  3.42150497e+00, ...,
           9.80752259e-02,  6.19966164e+01,  0.00000000e+00],
         ...,
         [ 1.08200000e+03,  2.29695053e+02,  3.13443232e+00, ...,
           3.12863648e-01,  6.39592133e+01,  0.00000000e+00],
         [ 1.27200000e+03,  2.30027084e+02,  3.12011576e+00, ...,
           3.12863648e-01,  6.40376892e+01,  0.00000000e+00],
         [ 1.35300000e+03,  2.30383743e+02,  3.10654211e+00, ...,
           3.12863648e-01,  6.40979309e+01,  0.00000000e+00]],

        [[ 1.59200000e+03,  2.26180832e+02,  3.40784287e+00, ...,
           6.54724836e-02,  6.21287651e+01,  0.00000000e+00],
         [ 1.78100000e+03,  2

In [19]:
labels[0]

<tf.Tensor: shape=(32, 32, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]], dtype=float32)>

## Evaluation Functions

##

In [34]:
import numpy as np
from sklearn.metrics import precision_recall_curve, auc, confusion_matrix, jaccard_score, mean_absolute_error
import cv2
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

def calc_auc_pr(y_pred_proba, y_true):
    """
    Args:
        y_pred_proba: predicted probabilities, shape (32,32,1)
        y_true: ground truth, shape (32,32,1)
    Returns:
        Single value from 0 to 1. Higher value (closer to 1) indicates the model successfully identifies fires
        while minimizing unnecessary false alarms 
    """
    y_pred_flat = y_pred_proba.numpy().flatten()
    y_true_flat = y_true.numpy().flatten()
    precision, recall, _ = precision_recall_curve(y_true_flat, y_pred_flat) #not using third output of thresholds
    return auc(recall, precision)

def calc_false_negative_rate(y_pred, y_true):
    """
    Args:
        y_pred: binary predictions, shape (32,32,1)
        y_true: ground truth, shape (32,32,1)
    Returns:
        Single value from 0 to 1. Lower value (closer to 0) indicates that the model predicts less false negatives. In this context,
        false negatives (inaccurately predicting no wildfire) is catastrophic and avoiding them should be a priority.
    """
    y_pred_flat = y_pred.numpy().flatten()
    y_true_flat = y_true.numpy().flatten()
    _, _, fn, tp = confusion_matrix(y_true_flat, y_pred_flat).ravel() #not using first two outputs of true negative and false positives
    return fn / (fn + tp + 1e-8) #avoid division by zero

def calc_jaccard(y_pred, y_true):
    """
    Args:
        y_pred: binary predictions, shape (32,32,1)
        y_true: ground truth, shape (32,32,1)
    Returns:
        Single value from 0 to 1. High value (closer to 1) indicates that the model better captures the overall extent
        and shape of the entire fire (value of 0.8 indicates predicted overlaps with 80% of actual)
    """
    y_pred_flat = y_pred.numpy().flatten()
    y_true_flat = y_true.numpy().flatten()
    return jaccard_score(y_true_flat, y_pred_flat)

def calc_mae_fire_front(y_pred, y_true, unmatched_penalty=50):
    """
    NOTE: can rewrite this if found to be too aggressive while evaluating models (since this does one-to-one, which penalizes the model
    heavily when it predicts a fire front pixel that doesn't actually exist in the true mask)
    Args:
        y_pred: binary predictions, shape (32,32,1)
        y_true: ground truth, shape (32,32,1)
        unmatched_penalty: penalty for when a predicted fire front pixel has no corresponding match in the true mask, lower if too aggressive
    Returns:
        Single value from 0 to inf (technically in this case, it's limited by the input grid size). A lower value (closer to 0) indicates
        that the predicted fire front is closer (avg number of pixels) to the actual fire front. This is important because understanding the
        fire front/leading edge of the fire is critical for firefighting, evacuation and resource allocation decisions. 
    """
    def extract_fire_front(mask):
        """
        Args:
            mask: binary 2D nparray
        Returns:
            coordinates of fire front pixels
        """
        mask = mask.numpy().squeeze().astype(np.uint8) #ensure 2D

        edges = cv2.Canny(mask * 255, 100, 200) #canny gets firefront using edge detection
        front_coords = np.column_stack(np.where(edges > 0)) #returns (rows, columns), ie (y,x)
        return front_coords
    
    pred_front = extract_fire_front(y_pred)
    true_front = extract_fire_front(y_true)

    if len(pred_front) == 0 or len(true_front) == 0:
        return np.nan #no front to calculate using
    
    #pad smaller away with dummy points far away to enforce one-to-one matching, penalizing inaccurate edges
    max_len = max(len(pred_front), len(true_front))
    #far point to penalize unmatched assignments
    dummy_point = np.array([[unmatched_penalty, unmatched_penalty]]) 

    if len(pred_front) < max_len:
        pad = np.tile(dummy_point, (max_len - len(pred_front), 1))
        pred_front = np.vstack([pred_front, pad])
    elif len(true_front) < max_len:
        pad = np.tile(dummy_point, (max_len - len(true_front), 1))
        true_front = np.vstack([true_front, pad])

    #calc pairwise distances
    cost_matrix = cdist(pred_front, true_front)

    #solve optimal one to one matching
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    matched_dists = cost_matrix[row_ind, col_ind]

    return np.mean(matched_dists)

#### Testing out the metrics

In [39]:
def evaluate(y_pred, y_pred_proba, y_true):
    print(f"AUC-PR: {calc_auc_pr(y_pred, y_true)}")
    print(f"FNR: {calc_false_negative_rate(y_pred, y_true)}")
    print(f"Jaccard Index: {calc_jaccard(y_pred, y_true)}")
    print(f"MAE Fire Front: {calc_mae_fire_front(y_pred, y_true)}")

In [40]:
#same map for pred and true, should return max values for everything

y_pred = labels[0]
y_pred_proba = tf.cast(y_pred, dtype=tf.float32)
y_true = labels[0]

evaluate(y_pred=y_pred, y_pred_proba=y_pred_proba, y_true=y_true)

AUC-PR: 1.0
FNR: 0.0
Jaccard Index: 1.0
MAE Fire Front: 0.0


In [41]:
#opposite (flipped) map for pred and true, should return min or at least low values for everything

y_pred = 1 - labels[0]
y_pred_proba = tf.cast(y_pred, dtype=tf.float32)
y_true = labels[0]

evaluate(y_pred=y_pred, y_pred_proba=y_pred_proba, y_true=y_true)
#will need to relook at MAE value here to see why it's zeroed out?

AUC-PR: 0.00341796875
FNR: 0.9999999985714286
Jaccard Index: 0.0
MAE Fire Front: 0.0


In [42]:
# random selection of two gold masks to see what happens
y_pred = labels[2]
y_pred_proba = tf.cast(y_pred, dtype=tf.float32)
y_true = labels[3]

evaluate(y_pred=y_pred, y_pred_proba=y_pred_proba, y_true=y_true)

AUC-PR: 0.017578125
FNR: 0.9999999997222222
Jaccard Index: 0.0
MAE Fire Front: 39.75219186998933
